In [1]:
using JuMP, Gurobi, XLSX

In [2]:
xf = XLSX.readxlsx("toy-stores.xlsx")

XLSXFile("toy-stores.xlsx") containing 1 Worksheet
            sheetname size          range        
-------------------------------------------------
               Sheet1 102x12        A1:L102      


In [3]:
data = xf["Sheet1"]

102×12 XLSX.Worksheet: ["Sheet1"](A1:L102)

In [4]:
model = Model(with_optimizer(Gurobi.Optimizer))
M = 100
N = 24

Academic license - for non-commercial use only


24

In [5]:
set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "Heuristics", 0)
set_optimizer_attribute(model, "Cuts", 0)

0

In [6]:
# Variables
@variable(model, 0 <= x[1:M,1:N]  <= 1) # + Constraint (4)
@variable(model, 0 <= y[1:N], Bin ) # + Constraint (5)

24-element Array{VariableRef,1}:
 y[1]
 y[2]
 y[3]
 y[4]
 y[5]
 y[6]
 y[7]
 y[8]
 y[9]
 y[10]
 y[11]
 y[12]
 y[13]
 y[14]
 y[15]
 y[16]
 y[17]
 y[18]
 y[19]
 y[20]
 y[21]
 y[22]
 y[23]
 y[24]

In [15]:
# Compute costs C_ij and fixed costs f_j
C = zeros(Float64, M,N)
f = zeros(Float64,N)
for j in 1:N
    lat2 = deg2rad(data[j+2,11])
    long2 = deg2rad(data[j+2,10])
    f[j]=data[j+2,12]
    for i in 1:M
        lat = deg2rad(data[i+2,4]) 
        long = deg2rad(data[i+2,3])
        if lat==lat2 && long ==long2 
            C[i,j]=0
        else
            C[i,j]= acos(sin(lat)*sin(lat2)+cos(lat)*cos(lat2)*cos(long-long2))*3958.8
        end
    end
end

110.9564983252387431
120.9564346379633937
130.9737255596032024
140.990423012373207
150.9989055144175452
160.9274544115454778
170.9660091704782416
180.9615052324952997
190.9395931661030517
1100.9865386791748685
1110.9862263883731699
1120.8541988951796333
1130.872677766113189
1140.9760071944147832
1150.8626805858149105
1160.8943394598251962
1170.991208043068212
1180.9057319238296533
1190.8698595083750306
1200.9741874283362342
1210.8970092026634457
1220.9929795430745518
1230.9044196519402521
1240.9960807026492096
220.994569957300752
230.9843276729373129
240.9070665223706214
250.9435908530875521
260.9960329278429221
270.9356366451858376
280.9910396881558332
290.9904549179762584
2100.9791029675713534
2110.9735483576841606
2120.9584638633070883
2130.9742630247046716
2140.9479432288353062
2150.9689377874768283
2160.9821314391553129
2170.9096215699435719
2180.9892131322456398
2190.9416197726428625
2200.9921235650803301
2210.9561575290801181
2220.9152950530788815
2230.9889008018668439
2240.9537

22220.9107530995946431
22230.9889446456149621
22240.9415414666624515
2310.9477771606581211
2320.9576846360816486
2330.979559998727048
2340.9881811668169891
2350.9927374476360051
2360.9196102673314901
2370.9851291964380824
2380.9652973000024577
2390.9433981420256499
23100.9698697116831856
23110.9923516309377742
23120.8253066483868433
23130.8522935900452563
23140.9908451704228918
23150.8406773963853307
23160.8736251402795844
23170.9896546368284129
23180.8908529808062169
23190.8300644350572322
23200.9752164040018453
23210.8608000230655871
23220.9905607083053813
23230.8898719711974197
23240.999816820863493
2410.9789357326355539
2420.9802617061594472
2430.9900365619626218
2440.9730505538799235
2450.989717359327384
2460.9581984804823209
2470.9728195354573688
2480.9833344660196318
2490.968284706252528
24100.9898131909560595
24110.9950158580812012
24120.8893001231778549
24130.9098052074843791
24140.982333934122174
24150.9007361060195749
24160.9270101955804458
24170.974653374260199
24180.938954

55120.9222091003092197
55130.9298317688922375
55140.9456854950766418
55150.9234112719362082
55160.9468675618977984
55170.960628042946478
55180.9499328772821137
55190.9387886944718883
55200.9763082967452761
55210.9571898638065015
55220.9647916113324635
55230.9485997753771733
55240.9734068228181559
5610.9735483576841606
5620.9848042239324384
5630.9968484101611563
5640.9628278263034111
5650.9779529379994505
5660.9560278485614824
5670.9897716382554274
5680.9897459599357883
5690.9766652313217519
56100.9714551536124707
56120.8684146494727103
56130.8975613741576645
56140.995019468152211
56150.8872300424396415
56160.9131546494663938
56170.965311723599037
56180.9308000080166706
56190.8555791718194511
56200.993875200723259
56210.8825626173787273
56220.9675788308216112
56230.9303496166325098
56240.9940958828400501
5710.9179168249951064
5720.9216768905159569
5730.9496842125434157
5740.9995324355975701
5750.9966694336076517
5760.8803256818784979
5770.9598249462300026
5780.9303053114893295
5790.9009

88240.9738052804828881
8910.9514450294060812
8920.972603581234722
8930.9905493814376661
8940.9543893934333254
8950.9639156167755837
8960.9335523370784008
8970.9988448756301483
8980.9804580540683133
8990.9663250670246711
89100.9445236648387187
89110.9952740630343935
89120.8243780818677515
89130.8616430979291722
89140.9997246799463714
89150.8495846168094987
89160.8775215606043354
89170.9574156372892914
89180.9009025398938861
89190.8029054023837596
89200.9825053968025953
89210.8337963186421882
89220.9585905655112859
89230.9007234699832635
89240.988798036281119
9010.9423679443420447
9020.9029191554243812
9030.8719602702792719
9040.8073139357977657
9050.8599291982992806
9060.9416850928261928
9070.7740964578163004
9080.8892428245891482
9090.8936287236234561
90100.9381963653121708
90110.8558429741948113
90120.9863373108915072
90130.9715715986066
90140.8000468879320854
90150.9732545923824407
90160.9765743104720204
90170.8077339370315042
90180.9624815716103392
90190.9999783360340355
90200.89675

In [1]:
# Objective function
@objective(model, Min, sum(x*transpose(C)) + y*transpose(f))

LoadError: LoadError: UndefVarError: @objective not defined
in expression starting at In[1]:2

In [11]:
# Constraint (2)

In [12]:
# Constraint (3)